<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR_Unit1/blob/main/Lesson_HF_RL_Unit1%2Bbonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ССЫЛКА НА ОРИГИНАЛ:

https://colab.research.google.com/github/huggingface/deep-rl-class/blob/master/notebooks/bonus-unit1/bonus-unit1.ipynb

@huggingface

@simoninithomas

# Бонусный блок 1: Давайте научим собаку Хагги 🐶 приносить палку

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="Bonus Unit 1Thumbnail" width="50%">

В этой тетради мы закрепим то, что узнали в первом разделе, **научив собаку Хагги приносить палку, а затем играть с ней прямо в вашем браузере**

⬇️ Вот пример того, чего **вы достигнете в конце блока.** ⬇️ (запустите ▶ , чтобы увидеть)

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### Окружающая среда 🎮

- Собака Хагги, окружающая среда, созданная [Thomas Simonini](https://twitter.com/ThomasSimonini) основанная на [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### Используемая библиотека 📚

- [MLAgents (Hugging Face version)](https://github.com/huggingface/ml-agents)

## Цели этого блокнота 🏆

В конце блокнота вы:

- Разберетесь с понятиями **пространство состояний, пространство действий и функция вознаграждения**, используемые для обучения Хагги.
- Научите **своего собственного Хагги** приносить палку.
- Получите возможность играть **со своим обученным Huggy прямо в вашем браузере**.



## Этот блокнот взят из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

В этом бесплатном курсе вы будете:

- 📖 Изучите глубокое обучение с подкреплением в **теории и практике**.
- 🧑‍💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL and Sample Factory 2.0.
- 🤖 Обучайте **агентов в уникальных условиях** 

И еще проверьте учебную 📚 программу 👉 https://simoninithomas.github.io/deep-rl-course

The best way to keep in touch is to join our discord server to exchange with the community and with us 👉🏻 https://discord.gg/ydHrjt3WP5

## Предпосылки 🏗️

Прежде чем погрузиться в блокнот, вам нужно:

🔲 📚 **Развить понимание основ обучения с подкреплением** (MC, TD, Rewards hypothesis...) выполняя блок 1

🔲 📚 **Прочитать введение к Huggy**, выполнив бонусный блок 1

## Установите графический процессор 💪
- Чтобы **ускорить обучение агента, мы будем использовать графический процессор**. Чтобы сделать это, перейдите к `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1" width="40%">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2" width="30%">

## Клонируйте репозиторий и установите зависимости 🔽

- Нам нужно клонировать репозиторий, который **содержит экспериментальную версию библиотеки, которая позволяет вам отправлять вашего обученного агента в концентратор (Hub).**

In [1]:
%%capture
# Клонирование этого конкретного репозитория (может занять 3 минуты)
!git clone https://github.com/huggingface/ml-agents/

In [2]:
%%capture
# Зайдите в репозиторий и установите пакет (может занять 3 минуты)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

## Загрузите и переместите zip-файл среды в `./trained-envs-executables/linux/`

- Исполняемый файл нашей среды находится в zip-файле.
- Нам нужно загрузить его и разместить на `./trained-envs-executables/linux/`

In [3]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF" -O ./trained-envs-executables/linux/Huggy.zip && rm -rf /tmp/cookies.txt

--2023-01-11 08:59:45--  https://docs.google.com/uc?export=download&confirm=t&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF
Resolving docs.google.com (docs.google.com)... 108.177.119.138, 108.177.119.113, 108.177.119.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.119.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jb0pb5rp4scgn6oeuukrqp4h6k4t5vle/1673427525000/15803371278684422230/*/1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF?e=download&uuid=e9ad69de-2a4f-40bd-a3aa-3cce6b7f04e7 [following]
--2023-01-11 08:59:46--  https://doc-0k-28-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jb0pb5rp4scgn6oeuukrqp4h6k4t5vle/1673427525000/15803371278684422230/*/1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF?e=download&uuid=e9ad69de-2a4f-40bd-a3aa-3cce6b7f04e7
Resolving doc-0k-28-docs.googleusercontent.com (doc-0k-28-docs.googleusercontent.com)... 108.177.126.

Загрузите файл Huggy.zip с https://drive.google.com/uc?export=download&id=1zv3M95ZJTWHUVOWT6ckq_cm98nft8gdF с помощью `wget`. Ознакомьтесь с полным решением для загрузки больших файлов из GDrive [здесь](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)

In [5]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Убедитесь, что ваш файл доступен

In [6]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Давайте кратко рассмотрим, как работает эта среда

### Пространство состояний: что "воспринимает" Хагги.

Хагги не "видит" свое окружение. Вместо этого мы предоставляем ему информацию об окружающей среде:

- Положение мишени (палки)
- Относительное положение Хагги и цели
- Ориентация его ног.

Учитывая всю эту информацию, Хагги **может решить, какие действия предпринять дальше, чтобы достичь своей цели**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="70%">


### Пространство действия: какие движения может делать Хагги
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="70%">

**Суставные двигатели приводят в движение ноги Хагги**. Это означает, что для достижения цели он должен **научиться правильно вращать суставные двигатели каждой из своих ног, чтобы двигаться**.

### Функция вознаграждения

Функция вознаграждения разработана таким образом, чтобы **Хагги выполнил свою цель**: принес палку.

Помните, что одной из основ обучения с подкреплением является *гипотеза вознаграждения*: цель может быть описана как **максимизация ожидаемого совокупного вознаграждения**.

Наша цель состоит в том, чтобы Хагги **шел к палке, но не слишком сильно вращался**. Следовательно, наша функция вознаграждения должна транслировать эту цель.

Наша функция вознаграждения:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy reward function" width="70%">

- *Бонус за ориентацию*: мы **вознаграждаем агента за то, что он приблизился к цели**.
- *Штраф по времени*: штраф по фиксированному времени, назначаемый за каждое действие, чтобы **заставить его добраться до цели как можно быстрее**.
- *Штраф за вращение*: мы наказываем Хагги, если **он слишком сильно вращается и поворачивается слишком быстро**.
- *Награда за достижение цели*: мы вознаграждаем Хагги за **достижение цели**.

## Проверьте конфигурационный файл Хагги

- В ML-Agents гиперпараметры обучения определяются  **в файле config.yaml.**

- В рамках этого блокнота мы не собираемся изменять гиперпараметры, но если вы хотите попробовать в качестве эксперимента, вам следует попробовать изменить некоторые гиперпараметры, Unity предоставляет очень [хорошую документацию, объясняющую каждый из них здесь](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

- **В случае, если вы хотите изменить гиперпараметры**, в записной книжке Google Colab вы можете открыть config.yaml: `/content/ml-agents/config/ppo/Huggy.yaml`

- Например, **если вы хотите сохранить больше моделей во время обучения** (на данный момент мы сохраняем каждые 200 000 временных шагов обучения). Вам нужно изменить:
  - `checkpoint_interval`: количество временных шагов обучения, между каждой контрольной точкой.
  - `keep_checkpoints`: максимальное количество контрольных точек для сохранения модели.

=> Просто имейте в виду, что **уменьшение значения `checkpoint_interval` означает больше моделей для загрузки в концентратор и, следовательно, более длительное время загрузки**

Теперь мы готовы обучить нашего агента 🔥 .

## Обучите нашего агента

Чтобы обучить нашего агента, нам просто нужно **запустить mlagents-learn и выбрать исполняемый файл, содержащий среду.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="70%">

 С ML Agents, мы запускаем обучающий сценарий. Мы определяем четыре параметра:

1. `mlagents-learn <config>`: путь, по которому находится конфигурационный файл гиперпараметров.
2. `--env`: где находится исполняемый файл среды.
3. `--run_id`: имя, которое вы хотите присвоить своему идентификатору тренировочного прогона.
4. `--no-graphics`: чтобы не запускать визуализацию во время тренировки.

Обучите модель и используйте флаг `--resume`, чтобы продолжить обучение в случае прерывания.

> При первом использовании `--resume` произойдет сбой, попробуйте запустить блок еще раз, чтобы обойти ошибку.

Тренировка займет от 30 до 45 минут в зависимости от вашей машины (не забудьте **настроить графический процессор**). ☕️

In [9]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy" --no-graphics


            ┐  ╖
        ╓╖╬│╡  ││╬╖╖
    ╓╖╬│││││┘  ╬│││││╬╖
 ╖╬│││││╬╜        ╙╬│││││╖╖                               ╗╗╗
 ╬╬╬╬╖││╦╖        ╖╬││╗╣╣╣╬      ╟╣╣╬    ╟╣╣╣             ╜╜╜  ╟╣╣
 ╬╬╬╬╬╬╬╬╖│╬╖╖╓╬╪│╓╣╣╣╣╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╒╣╣╖╗╣╣╣╗   ╣╣╣ ╣╣╣╣╣╣ ╟╣╣╖   ╣╣╣
 ╬╬╬╬┐  ╙╬╬╬╬│╓╣╣╣╝╜  ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╣╙ ╙╣╣╣  ╣╣╣ ╙╟╣╣╜╙  ╫╣╣  ╟╣╣
 ╬╬╬╬┐     ╙╬╬╣╣      ╫╣╣╣╬      ╟╣╣╬    ╟╣╣╣ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣     ╣╣╣┌╣╣╜
 ╬╬╬╜       ╬╬╣╣      ╙╝╣╣╬      ╙╣╣╣╗╖╓╗╣╣╣╜ ╟╣╣╬   ╣╣╣  ╣╣╣  ╟╣╣╦╓    ╣╣╣╣╣
 ╙   ╓╦╖    ╬╬╣╣   ╓╗╗╖            ╙╝╣╣╣╣╝╜   ╘╝╝╜   ╝╝╝  ╝╝╝   ╙╣╣╣    ╟╣╣╣
   ╩╬╬╬╬╬╬╦╦╬╬╣╣╗╣╣╣╣╣╣╣╝                                             ╫╣╣╣╣
      ╙╬╬╬╬╬╬╬╣╣╣╣╣╣╝╜
          ╙╬╬╬╣╣╣╜
             ╙
        
 Version information:
  ml-agents: 0.29.0.dev0,
  ml-agents-envs: 0.29.0.dev0,
  Communicator API: 1.5.0,
  PyTorch: 1.8.1+cu102
[INFO] Connected to Unity environment with package version 2.2.1-exp.1 and communication version 1.5.0
[INFO] Connected new brain: Huggy?team=0


## Перенос агента на 🤗 Hub

- Теперь, когда мы обучили нашего агента, мы **готовы перенести его на хаб, чтобы иметь возможность играть с Huggy в браузере 🔥.**

Чтобы иметь возможность поделиться своей моделью с сообществом, необходимо выполнить еще три шага:

1️⃣ (Если это еще не сделано) создайте учетную запись для HF ➡ https://huggingface.co/join

2️⃣ Войдите в систему, а затем вам нужно сохранить свой токен аутентификации с веб-сайта Hugging Face.
- Создайте новый токен (https://huggingface.co/settings/tokens) **с разрешением на запись**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token" width="50%">

- Скопируйте токен 
- Запустите ячейку ниже и вставьте токен

In [10]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


Если вы не хотите использовать записную книжку Google Colabora Jupyter, вам нужно вместо этого использовать эту команду: `huggingface-cli login`

Или:

`from huggingface_hub import login`

`login(token=token, add_to_git_credential=True)`




Затем нам просто нужно запустить `mlagents-push-to-hf`.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mlpush.png" alt="ml learn function" width="70%">

И мы определяем 4 параметра:

1. `--run-id`: имя идентификатора тренировочного прогона.
2. `--local-dir`: там, где был сохранен агент, это `results/--run-id`, так что в нашем случае `results/Huggy`.
3. `--repo-id`: название репозитория Hugging Face, которое вы хотите создать или обновить. Это your `"huggingface username"/"the repo name"`. Если репозиторий не существует **, он будет создан автоматически**
4. `--commit-message`: поскольку репозитории HF являются репозиторием git, вам необходимо определить сообщение о фиксации.

In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy" --repo-id="Art-phys/ppo-Huggy" --commit-message="Huggy"

Если все сработало, вы должны получить это в конце процесса (но с вашим URL-адресом 😆) :



```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/ThomasSimonini/ppo-Huggy
```

Это ссылка на ваш репозиторий моделей. Репозиторий содержит карточку модели, которая объясняет, как использовать модель, ваши журналы Tensorboard и ваш конфигурационный файл. **это репозиторий git, что означает, что вы можете иметь разные коммиты, обновлять свой репозиторий новым нажатием, открывать запросы на извлечение и т.д.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn function" width="100%">

Теперь приступаем к самому интересному:: **возможность играть с Huggy онлайн 👀.**

## Поиграй со своим Huggy 🐕

Этот шаг самый простой:

- Откройте игру Huggy в своем браузере: https://singularite.itch.io/huggy

- Нажмите на `"Load your Huggy model"`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="70%">

1. На шаге 1 выберите репозиторий вашей модели, который является идентификатором модели (в примере ThomasSimonini/ppo-Huggy).

2. На шаге 2 **выберите, какую модель вы хотите воспроизвести**:
  - У меня их несколько, так как мы сохраняли модель каждые 500000 временных шагов.
  - Но так как я хочу более свежую, я выбираю `Huggy.onnx`

👉Что приятно **,так это попробовать моделями с различными шагами, чтобы видеть улучшение агента.**

Поздравляю с завершением этого бонусного блока!

Теперь вы можете сидеть и наслаждаться игрой со своим Huggy 🐶 . И не ** забудьте поделиться любовью, поделившись Huggy со своими друзьями 🤗 **. И если вы поделитесь этим в социальных сетях, ** пожалуйста, отметьте нас @huggingface и меня @simoninithomas**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="Huggy cover" width="50%">


## Keep Learning, Stay  awesome 🤗 Продолжай учиться